<a href="https://colab.research.google.com/github/motoroko/Hatespeach_Detection/blob/main/Hatespeach_Detection_using_Hybrid_NB_SVM_MLP_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.text import Text
from nltk.tokenize import word_tokenize

import re
import string

!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df = pd.read_csv('Data_Hatespeach.csv')
df

,Label,Tweet
0,HS,RT @Ardiesuhardi1: @CintaNirmala2 @IrmansyahAd...
1,HS,RT @Ardiesuhardi1: @CintaNirmala2 @Bianglala_0...
2,HS,RT @HaneutM: Semoga..\n#21AprilJokowiTumbang
3,HS,RT @dancersejati1: JAKSA AGUNG MANTAP!\nKorupt...
4,HS,RT @Ardiesuhardi1: @banditBirokrat @Sangkurian...
...,...,...
4428,HS,RT @DimasAnugrahe: Terimakasih pada semua piha...
4429,HS,RT @CantikaAndiP: Terimakasih pada semua pihak...
4430,HS,RT @CantikaAndiP: Terimakasih pada semua pihak...
4431,Non_HS,RT @JoniSaputra00: Aamiin!! semoga lancar samp...


In [ ]:
def clean_text(text):
  text = text.lower() #lowercase atau case folding
  text = re.sub('@[^\s]+', '', text) #remove username
  text = re.sub('\[.*?\]', '', text) # remove square brackets
  text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', text) # remove URLs
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # remove punctuation
  text = re.sub('\w*\d\w*', '', text)
  text = re.sub('[‘’“”…]', '', text)
  text = re.sub('\n', '', text)
  return text

clean1 = lambda x: clean_text(x)

In [ ]:
df['Tweet'] = pd.DataFrame(df['Tweet'].apply(clean1))

In [ ]:
from sklearn.preprocessing import LabelEncoder
Class_encoder = LabelEncoder().fit_transform(df.Label)
unique, counts = np.unique(Class_encoder, return_counts=True)

print(np.asarray((unique, counts)))

[[   0    1]
 [3291 1142]]


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
stop_words = stopwords.words('indonesian')
CountVectorizer = CountVectorizer(min_df=5, max_df=0.7, ngram_range = (1,5), stop_words=stop_words)
df_vector = CountVectorizer.fit_transform(df.Tweet).toarray()
df_vector = pd.DataFrame(df_vector)
df_vector

,0,1,2,3,4,5,6,7,8,9,...,2832,2833,2834,2835,2836,2837,2838,2839,2840,2841
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4428,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4429,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4430,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4431,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Model


In [ ]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
kf.get_n_splits(df_vector)

5

## NB Model

### Parameter Tuning

In [ ]:
parameters = {
    'alpha':list(np.arange(0.1,2.1,0.1))
}
MNB = MultinomialNB()
clf = GridSearchCV(MNB, parameters, verbose=3, cv=2)
clf.fit(df_vector, Class_encoder)

Fitting 2 folds for each of 20 candidates, totalling 40 fits
[CV 1/2] END .........................alpha=0.1;, score=0.645 total time=   0.2s
[CV 2/2] END .........................alpha=0.1;, score=0.683 total time=   0.2s
[CV 1/2] END .........................alpha=0.2;, score=0.636 total time=   0.2s
[CV 2/2] END .........................alpha=0.2;, score=0.683 total time=   0.2s
[CV 1/2] END .........alpha=0.30000000000000004;, score=0.634 total time=   0.2s
[CV 2/2] END .........alpha=0.30000000000000004;, score=0.670 total time=   0.2s
[CV 1/2] END .........................alpha=0.4;, score=0.628 total time=   0.2s
[CV 2/2] END .........................alpha=0.4;, score=0.664 total time=   0.2s
[CV 1/2] END .........................alpha=0.5;, score=0.628 total time=   0.2s
[CV 2/2] END .........................alpha=0.5;, score=0.664 total time=   0.2s
[CV 1/2] END .........................alpha=0.6;, score=0.624 total time=   0.2s
[CV 2/2] END .........................alpha=0.6;

GridSearchCV(cv=2, estimator=MultinomialNB(),
             param_grid={'alpha': [0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6,
                                   0.7000000000000001, 0.8, 0.9, 1.0, 1.1,
                                   1.2000000000000002, 1.3000000000000003,
                                   1.4000000000000001, 1.5000000000000002, 1.6,
                                   1.7000000000000002, 1.8000000000000003,
                                   1.9000000000000001, 2.0]},
             verbose=3)

In [ ]:
print('parameter terbaik : ',clf.best_params_)
print('Akurasi parameter : ',clf.best_score_)

parameter terbaik :  {'alpha': 0.1}
Akurasi parameter :  0.663888759975835


### Cross Validation

In [ ]:
n_fold = 1
akurasi = []
for train_index, test_index in kf.split(df_vector):
  print('Fold ke = ',n_fold)
  x_train, x_test = df_vector.iloc[train_index], df_vector.iloc[test_index]
  y_train, y_test = Class_encoder[train_index], Class_encoder[test_index]

  NB_model = MultinomialNB(**clf.best_params_)
  NB_model.fit(x_train,y_train)
  NB_pred = NB_model.predict(x_test)

  print(classification_report(y_test,NB_pred))

  unique, counts = np.unique((y_test == NB_pred.reshape(NB_pred.shape[0],)), return_counts=True)
  acc = (counts[1]/np.sum(counts)) * 100
  akurasi.append(acc)
  n_fold += 1
  print(f'Accuracy : {acc:.3f} %'.format(acc))
  print('-'*50)

print(f'Rata-rata akurasi metode : {np.mean(akurasi):.3f} %'.format(np.mean(akurasi)))

Fold ke =  1
              precision    recall  f1-score   support

           0       0.93      0.78      0.85       656
           1       0.58      0.84      0.69       231

    accuracy                           0.80       887
   macro avg       0.76      0.81      0.77       887
weighted avg       0.84      0.80      0.81       887

Accuracy : 79.932 %
--------------------------------------------------
Fold ke =  2
              precision    recall  f1-score   support

           0       0.93      0.80      0.86       642
           1       0.62      0.85      0.72       245

    accuracy                           0.81       887
   macro avg       0.78      0.83      0.79       887
weighted avg       0.85      0.81      0.82       887

Accuracy : 81.398 %
--------------------------------------------------
Fold ke =  3
              precision    recall  f1-score   support

           0       0.95      0.80      0.87       666
           1       0.59      0.87      0.70       221

 

## SVM Model


### Parameter Tuning

In [ ]:
parameters = {
    'C':[0.01,0.1,1,10,100],
    'kernel':['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': list(np.arange(1,6,1)),
    'gamma': ['scale', 'auto'],
}
svc = SVC()
clf = GridSearchCV(svc, parameters, verbose=3, cv=2)
clf.fit(df_vector, Class_encoder)

Fitting 2 folds for each of 200 candidates, totalling 400 fits
[CV 1/2] END C=0.01, degree=1, gamma=scale, kernel=linear;, score=0.548 total time=   9.1s
[CV 2/2] END C=0.01, degree=1, gamma=scale, kernel=linear;, score=0.744 total time=  14.3s
[CV 1/2] END C=0.01, degree=1, gamma=scale, kernel=poly;, score=0.742 total time=  14.4s
[CV 2/2] END C=0.01, degree=1, gamma=scale, kernel=poly;, score=0.742 total time=  22.3s
[CV 1/2] END C=0.01, degree=1, gamma=scale, kernel=rbf;, score=0.584 total time=  18.0s
[CV 2/2] END C=0.01, degree=1, gamma=scale, kernel=rbf;, score=0.742 total time=  23.5s
[CV 1/2] END C=0.01, degree=1, gamma=scale, kernel=sigmoid;, score=0.742 total time=  15.5s
[CV 2/2] END C=0.01, degree=1, gamma=scale, kernel=sigmoid;, score=0.742 total time=  13.2s
[CV 1/2] END C=0.01, degree=1, gamma=auto, kernel=linear;, score=0.548 total time=   9.5s
[CV 2/2] END C=0.01, degree=1, gamma=auto, kernel=linear;, score=0.744 total time=  16.4s
[CV 1/2] END C=0.01, degree=1, gamma=

GridSearchCV(cv=2, estimator=SVC(),
             param_grid={'C': [0.01, 0.1, 1, 10, 100],
                         'degree': [1, 2, 3, 4, 5], 'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             verbose=3)

In [ ]:
print('parameter terbaik : ',clf.best_params_)
print('Akurasi parameter : ',clf.best_score_)

parameter terbaik :  {'C': 0.1, 'degree': 1, 'gamma': 'scale', 'kernel': 'sigmoid'}
Akurasi parameter :  0.748927002372543


### Cross Validation

In [ ]:
n_fold = 1
akurasi = []
for train_index, test_index in kf.split(df_vector):
  print('Fold ke = ',n_fold)
  x_train, x_test = df_vector.iloc[train_index], df_vector.iloc[test_index]
  y_train, y_test = Class_encoder[train_index], Class_encoder[test_index]

  SVC_model = SVC(**clf.best_params_)
  SVC_model.fit(x_train,y_train)
  SVC_pred = SVC_model.predict(x_test)

  print(classification_report(y_test,SVC_pred))

  unique, counts = np.unique((y_test == SVC_pred.reshape(SVC_pred.shape[0],)), return_counts=True)
  acc = (counts[1]/np.sum(counts)) * 100
  akurasi.append(acc)
  n_fold += 1
  print(f'Accuracy : {acc:.3f} %'.format(acc))
  print('-'*50)

print(f'Rata-rata akurasi metode : {np.mean(akurasi):.3f} %'.format(np.mean(akurasi)))

Fold ke =  1
              precision    recall  f1-score   support

           0       0.74      1.00      0.85       656
           1       0.00      0.00      0.00       231

    accuracy                           0.74       887
   macro avg       0.37      0.50      0.43       887
weighted avg       0.55      0.74      0.63       887

Accuracy : 73.957 %
--------------------------------------------------
Fold ke =  2
              precision    recall  f1-score   support

           0       0.72      1.00      0.84       642
           1       0.00      0.00      0.00       245

    accuracy                           0.72       887
   macro avg       0.36      0.50      0.42       887
weighted avg       0.52      0.72      0.61       887

Accuracy : 72.379 %
--------------------------------------------------
Fold ke =  3
              precision    recall  f1-score   support

           0       0.75      1.00      0.86       666
           1       1.00      0.00      0.01       221

 

## MLP Model

### Parameter Tuning

In [ ]:
parameters = {
    'activation':['identity', 'logistic', 'tanh', 'relu'],
    'solver':['lbfgs', 'sgd', 'adam'],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
}
mlp = MLPClassifier()
clf = GridSearchCV(mlp, parameters, verbose=3, cv=2)
clf.fit(df_vector, Class_encoder)

Fitting 2 folds for each of 200 candidates, totalling 400 fits
[CV 1/2] END C=0.01, degree=1, gamma=scale, kernel=linear;, score=0.548 total time=   9.1s


In [ ]:
print('parameter terbaik : ',clf.best_params_)
print('Akurasi parameter : ',clf.best_score_)

### Cross Validation

In [ ]:
n_fold = 1
akurasi = []
for train_index, test_index in kf.split(df_vector):
  print('Fold ke = ',n_fold)
  x_train, x_test = df_vector.iloc[train_index], df_vector.iloc[test_index]
  y_train, y_test = Class_encoder[train_index], Class_encoder[test_index]

  MLP_model = MLPClassifier(random_state=42, **clf.best_params_)
  MLP_model.fit(x_train,y_train)
  MLP_pred = MLP_model.predict(x_test)

  print(classification_report(y_test,MLP_pred))

  unique, counts = np.unique((y_test == MLP_pred.reshape(MLP_pred.shape[0],)), return_counts=True)
  acc = (counts[1]/np.sum(counts)) * 100
  akurasi.append(acc)
  n_fold += 1
  print(f'Accuracy : {acc:.3f} %'.format(acc))
  print('-'*50)

print(f'Rata-rata akurasi metode : {np.mean(akurasi):.3f} %'.format(np.mean(akurasi)))

Fold ke =  1
              precision    recall  f1-score   support

           0       0.83      0.92      0.88       656
           1       0.69      0.48      0.57       231

    accuracy                           0.81       887
   macro avg       0.76      0.70      0.72       887
weighted avg       0.80      0.81      0.80       887

Accuracy : 80.834 %
--------------------------------------------------
Fold ke =  2
              precision    recall  f1-score   support

           0       0.83      0.94      0.88       642
           1       0.76      0.49      0.60       245

    accuracy                           0.82       887
   macro avg       0.79      0.72      0.74       887
weighted avg       0.81      0.82      0.80       887

Accuracy : 81.623 %
--------------------------------------------------
Fold ke =  3
              precision    recall  f1-score   support

           0       0.88      0.94      0.91       666
           1       0.78      0.60      0.68       221

 

## NB-SVM Model

In [ ]:
class NBSVMClassifier(BaseEstimator, ClassifierMixin):
  def __init__(self, beta=0.5, alpha=1, C=1.0, kernel='linear', degree=3 , gamma='auto',
                class_weight='balanced'):
    self.C = C
    self.kernel = kernel
    self.degree = degree
    self.gamma = gamma
    self.beta = beta
    self.alpha = alpha
    self.r = None
    self.coef_ = None
    self.intercept_ = None
    self.interpolated_coef_ = None
    self.interpolated_intercept_ = None
    self.class_weight = class_weight

  def _log_probs(self, X, y):
    p_sum = np.sum(X[y==1], axis=0) + self.alpha
    n_sum = np.sum(X[y==0], axis=0) + self.alpha
    p_tot = len(X[y==1]) + self.alpha
    n_tot = len(X[y==0]) + self.alpha
    p_ratio = p_sum/p_tot
    n_ratio = n_sum/n_tot
    self.r = np.log(p_ratio/n_ratio)
    return self

  def _binarize(self, X):
    X[X>0] = 1
    return X

  def _interpolate(self, weights):
    return (self.beta*weights) + (1-self.beta)*(np.mean(abs(weights)))

  def fit(self, X, y):
    X_binarized = X.copy()
    X_binarized = self._binarize(X_binarized)
    self._log_probs(X_binarized, y)
    del X_binarized
    X_nb = X*self.r
    svm = SVC(C=self.C, kernel=self.kernel, degree=self.degree, gamma=self.gamma, class_weight=self.class_weight)
    svm.fit(X_nb, y)
    self.coef_ = svm.coef_
    self.intercept_ = svm.intercept_
    self.interpolated_coef_ = self._interpolate(self.coef_)
    self.interpolated_intercept_ = self._interpolate(self.intercept_)
    return self

  def predict(self, X):
    X_nb = X*self.r
    preds = np.add(np.dot(X_nb, self.interpolated_coef_.transpose()), self.interpolated_intercept_)
    preds[preds>0] = 1
    preds[preds<=0] = 0
    return preds

### Parameter Tuning

In [ ]:
parameters = {
    'alpha':list(np.arange(0.1,2.1,0.1)),
    'C':[0.01,0.1,1,10,100],
    'kernel':['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': list(np.arange(1,6,1)),
    'gamma': ['scale', 'auto'],
}
nbsvm = NBSVMClassifier()
clf = GridSearchCV(nbsvm, parameters, verbose=3, cv=2)
clf.fit(df_vector, Class_encoder)

In [ ]:
print('parameter terbaik : ',clf.best_params_)
print('Akurasi parameter : ',clf.best_score_)

### Cross Validation

In [ ]:
n_fold = 1
akurasi = []
for train_index, test_index in kf.split(df_vector):
  print('Fold ke = ',n_fold)
  x_train, x_test = df_vector.iloc[train_index], df_vector.iloc[test_index]
  y_train, y_test = Class_encoder[train_index], Class_encoder[test_index]

  NBSVM_model = NBSVMClassifier(**clf.best_params_)
  NBSVM_model.fit(x_train,y_train)
  NBSVM_pred = NBSVM_model.predict(x_test)

  print(classification_report(y_test,NBSVM_pred))

  unique, counts = np.unique((y_test == NBSVM_pred.reshape(NBSVM_pred.shape[0],)), return_counts=True)
  acc = (counts[1]/np.sum(counts)) * 100
  akurasi.append(acc)
  n_fold += 1
  print(f'Accuracy : {acc:.3f} %'.format(acc))
  print('-'*50)

print(f'Rata-rata akurasi metode : {np.mean(akurasi):.3f} %'.format(np.mean(akurasi)))

Fold ke =  1
              precision    recall  f1-score   support

           0       0.99      0.72      0.83       656
           1       0.55      0.97      0.70       231

    accuracy                           0.78       887
   macro avg       0.77      0.84      0.76       887
weighted avg       0.87      0.78      0.80       887

Accuracy : 78.241 %
--------------------------------------------------
Fold ke =  2
              precision    recall  f1-score   support

           0       0.99      0.73      0.84       642
           1       0.58      0.97      0.73       245

    accuracy                           0.80       887
   macro avg       0.78      0.85      0.78       887
weighted avg       0.87      0.80      0.81       887

Accuracy : 79.932 %
--------------------------------------------------
Fold ke =  3
              precision    recall  f1-score   support

           0       0.99      0.73      0.84       666
           1       0.55      0.98      0.70       221

 

## NB-MLP Model

In [ ]:
class NBMLPClassifier(BaseEstimator, ClassifierMixin):
  def __init__(self, beta=0.5, alpha=1, activation='relu', solver='adam' ,learning_rate='constant',
              class_weight='balanced'):
    self.activation = activation
    self.solver = solver
    self.learning_rate = learning_rate
    self.beta = beta
    self.alpha = alpha
    self.r = None
    self.coef_ = None
    self.intercept_ = None
    self.interpolated_coef_ = None
    self.interpolated_intercept_ = None

  def _log_probs(self, X, y):
    p_sum = np.sum(X[y==1], axis=0) + self.alpha
    n_sum = np.sum(X[y==0], axis=0) + self.alpha
    p_tot = len(X[y==1]) + self.alpha
    n_tot = len(X[y==0]) + self.alpha
    p_ratio = p_sum/p_tot
    n_ratio = n_sum/n_tot
    self.r = np.log(p_ratio/n_ratio)
    return self

  def _binarize(self, X):
    X[X>0] = 1
    return X

  def _interpolate(self, weights):
    return (self.beta*weights) + (1-self.beta)*(np.mean(abs(weights)))

  def fit(self, X, y):
    X_binarized = X.copy()
    X_binarized = self._binarize(X_binarized)
    self._log_probs(X_binarized, y)
    del X_binarized
    X_nb = X*self.r
    mlp = MLPClassifier(random_state=42, learning_rate=self.learning_rate, solver=self.solver,
                            activation=self.activation)
    mlp.fit(X_nb, y)
    self.coef_ = mlp.coefs_[0].sum(axis=1)
    self.intercept_ = mlp.intercepts_[1]
    self.interpolated_coef_ = self._interpolate(self.coef_)
    self.interpolated_intercept_ = self._interpolate(self.intercept_)
    return self

  def predict(self, X):
    X_nb = X*self.r
    preds = np.add(np.dot(X_nb, self.interpolated_coef_.transpose()), self.interpolated_intercept_)
    preds[preds>0] = 1
    preds[preds<=0] = 0
    return preds

### Parameter Tuning

In [ ]:
parameters = {
    'alpha':list(np.arange(0.1,2.1,0.1)),
    'activation':['identity', 'logistic', 'tanh', 'relu'],
    'solver':['lbfgs', 'sgd', 'adam'],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
}
nbmlp = NBMLPClassifier()
clf = GridSearchCV(nbmlp, parameters, verbose=3, cv=2)
clf.fit(df_vector, Class_encoder)

In [ ]:
print('parameter terbaik : ',clf.best_params_)
print('Akurasi parameter : ',clf.best_score_)

### Cross Validation

In [ ]:
n_fold = 1
akurasi = []
for train_index, test_index in kf.split(df_vector):
  print('Fold ke = ',n_fold)
  x_train, x_test = df_vector.iloc[train_index], df_vector.iloc[test_index]
  y_train, y_test = Class_encoder[train_index], Class_encoder[test_index]

  NBMLP_model = NBMLPClassifier(**clf.best_params_)
  NBMLP_model.fit(x_train,y_train)
  NBMLP_pred = NBMLP_model.predict(x_test)

  print(classification_report(y_test,NBMLP_pred))

  unique, counts = np.unique((y_test == NBMLP_pred.reshape(NBMLP_pred.shape[0],)), return_counts=True)
  acc = (counts[1]/np.sum(counts)) * 100
  akurasi.append(acc)
  n_fold += 1
  print(f'Accuracy : {acc:.3f} %'.format(acc))
  print('-'*50)

print(f'Rata-rata akurasi metode : {np.mean(akurasi):.3f} %'.format(np.mean(akurasi)))

Fold ke =  1
              precision    recall  f1-score   support

           0       0.88      0.76      0.82       656
           1       0.51      0.71      0.60       231

    accuracy                           0.75       887
   macro avg       0.70      0.74      0.71       887
weighted avg       0.79      0.75      0.76       887

Accuracy : 74.859 %
--------------------------------------------------
Fold ke =  2
              precision    recall  f1-score   support

           0       0.89      0.77      0.82       642
           1       0.55      0.74      0.63       245

    accuracy                           0.76       887
   macro avg       0.72      0.75      0.73       887
weighted avg       0.79      0.76      0.77       887

Accuracy : 76.212 %
--------------------------------------------------
Fold ke =  3
              precision    recall  f1-score   support

           0       0.89      0.75      0.81       666
           1       0.49      0.73      0.59       221

 

## NB-SVM-MLP Model

In [ ]:
class NBMLPSVMClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, beta=0.5, alpha=1, activation='relu', solver='adam' ,learning_rate='constant',
                 C=1.0, kernel='linear', degree=3 , gamma='auto',
                 class_weight='balanced'):
        self.C = C
        self.kernel = kernel
        self.degree = degree
        self.gamma = gamma
        self.activation = activation
        self.solver = solver
        self.learning_rate = learning_rate
        self.beta = beta
        self.alpha = alpha
        self.r = None
        self.coef_ = None
        self.intercept_ = None
        self.interpolated_coef_ = None
        self.interpolated_intercept_ = None
        self.class_weight = class_weight

    def _log_probs(self, X, y):
        p_sum = np.sum(X[y==1], axis=0) + self.alpha
        n_sum = np.sum(X[y==0], axis=0) + self.alpha
        p_tot = len(X[y==1]) + self.alpha
        n_tot = len(X[y==0]) + self.alpha
        p_ratio = p_sum/p_tot
        n_ratio = n_sum/n_tot
        self.r = np.log(p_ratio/n_ratio)
        return self

    def _binarize(self, X):
        X[X>0] = 1
        return X

    def _interpolate(self, weights):
        return (self.beta*weights) + (1-self.beta)*(np.mean(abs(weights)))

    def fit(self, X, y):
        X_binarized = X.copy()
        X_binarized = self._binarize(X_binarized)
        self._log_probs(X_binarized, y)
        del X_binarized
        X_nb = X*self.r
        svm = SVC(C=self.C, kernel=self.kernel, degree=self.degree, gamma=self.gamma, class_weight=self.class_weight)
        svm.fit(X_nb, y)
        mlp = MLPClassifier(random_state=42, learning_rate=self.learning_rate, solver=self.solver,
                            activation=self.activation)
        mlp.fit(X, y)
        self.coef_ = np.add(svm.coef_ , mlp.coefs_[0].sum(axis=1))
        self.intercept_ = np.add(svm.intercept_, mlp.intercepts_[1])
        self.interpolated_coef_ = self._interpolate(self.coef_)
        self.interpolated_intercept_ = self._interpolate(self.intercept_)

        return self

    def predict(self, X):
        X_nb = X*self.r
        preds = np.add(np.dot(X_nb, self.interpolated_coef_.transpose()), self.interpolated_intercept_)
        preds[preds>0] = 1
        preds[preds<=0] = 0
        return preds

### Parameter Tuning

In [ ]:
parameters = {
    'alpha':list(np.arange(0.1,2.1,0.1)),
    'activation':['identity', 'logistic', 'tanh', 'relu'],
    'solver':['lbfgs', 'sgd', 'adam'],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'C':[0.01,0.1,1,10,100],
    'kernel':['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': list(np.arange(1,6,1)),
    'gamma': ['scale', 'auto'],
}
nbsvmmlp = NBMLPSVMClassifier()
clf = GridSearchCV(nbsvmmlp, parameters, verbose=3, cv=2)
clf.fit(df_vector, Class_encoder)

In [ ]:
print('parameter terbaik : ',clf.best_params_)
print('Akurasi parameter : ',clf.best_score_)

### Cross Validation

In [ ]:
n_fold = 1
akurasi = []
for train_index, test_index in kf.split(df_vector):
  print('Fold ke = ',n_fold)
  x_train, x_test = df_vector.iloc[train_index], df_vector.iloc[test_index]
  y_train, y_test = Class_encoder[train_index], Class_encoder[test_index]

  NBMLPSVM_model = NBMLPSVMClassifier(**clf.best_params_)
  NBMLPSVM_model.fit(x_train,y_train)
  NBMLPSVM_pred = NBMLPSVM_model.predict(x_test)

  print(classification_report(y_test,NBMLPSVM_pred))

  unique, counts = np.unique((y_test == NBMLPSVM_pred.reshape(NBMLPSVM_pred.shape[0],)), return_counts=True)
  acc = (counts[1]/np.sum(counts)) * 100
  akurasi.append(acc)
  n_fold += 1
  print(f'Accuracy : {acc:.3f} %'.format(acc))
  print('-'*50)

print(f'Rata-rata akurasi metode : {np.mean(akurasi):.3f} %'.format(np.mean(akurasi)))

Fold ke =  1
              precision    recall  f1-score   support

           0       0.96      0.75      0.84       656
           1       0.56      0.90      0.69       231

    accuracy                           0.79       887
   macro avg       0.76      0.82      0.76       887
weighted avg       0.85      0.79      0.80       887

Accuracy : 78.805 %
--------------------------------------------------
Fold ke =  2
              precision    recall  f1-score   support

           0       0.95      0.77      0.85       642
           1       0.60      0.90      0.72       245

    accuracy                           0.80       887
   macro avg       0.77      0.83      0.78       887
weighted avg       0.85      0.80      0.81       887

Accuracy : 80.383 %
--------------------------------------------------
Fold ke =  3
              precision    recall  f1-score   support

           0       0.98      0.74      0.84       666
           1       0.55      0.95      0.70       221

 